In [1]:
%matplotlib inline
import pandas as pd
from path import Path
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('./Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
columns = ["application_type",'next_pymnt_d','initial_list_status','pymnt_plan','loan_status','issue_d','verification_status','home_ownership']
df[columns]

,application_type,next_pymnt_d,initial_list_status,pymnt_plan,loan_status,issue_d,verification_status,home_ownership
0,Individual,May-2019,w,n,low_risk,Mar-2019,Source Verified,RENT
1,Individual,May-2019,w,n,low_risk,Mar-2019,Verified,MORTGAGE
2,Individual,May-2019,w,n,low_risk,Mar-2019,Verified,MORTGAGE
3,Individual,May-2019,w,n,low_risk,Mar-2019,Verified,RENT
4,Individual,May-2019,w,n,low_risk,Mar-2019,Not Verified,MORTGAGE
...,...,...,...,...,...,...,...,...
68812,Individual,May-2019,w,n,low_risk,Jan-2019,Source Verified,RENT
68813,Individual,May-2019,w,n,low_risk,Jan-2019,Not Verified,RENT
68814,Individual,May-2019,w,n,low_risk,Jan-2019,Source Verified,MORTGAGE
68815,Individual,May-2019,f,n,low_risk,Jan-2019,Verified,MORTGAGE


In [6]:
# risk dictionary
risk = {
    "low_risk": 0,
    "high_risk":1
}
# Months' names encoded using the dictionary values
df["loan_status"] = df["loan_status"].apply(lambda x: risk[x])
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,0,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,0,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,0,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,0,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,0,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2 = df.copy()
#columns1 = ["application_type",'initial_list_status','pymnt_plan','loan_status','verification_status','home_ownership']
df2['application_type'] = le.fit_transform(df2['application_type'])
df2['initial_list_status'] = le.fit_transform(df2['initial_list_status']) 
df2['pymnt_plan'] = le.fit_transform(df2['pymnt_plan']) 
df2['verification_status'] = le.fit_transform(df2['verification_status'])
df2['home_ownership'] = le.fit_transform(df2['home_ownership'])
df2['hardship_flag'] = le.fit_transform(df2['hardship_flag'])
df2['debt_settlement_flag'] = le.fit_transform(df2['debt_settlement_flag'])
df2.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,Mar-2019,0,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,Mar-2019,0,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,Mar-2019,0,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,Mar-2019,0,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,Mar-2019,0,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


In [8]:
columns2 = ['next_pymnt_d','issue_d']
loan_df = df2.drop(columns=columns2)

# Split the Data into Training and Testing

In [9]:
# Create our features
X =loan_df.drop(columns ='loan_status')

# Create our target
y = loan_df['loan_status']

In [10]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.0,21.778153,0.217766,0.497697,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.0,20.199244,0.718367,0.758122,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.0,0.000000,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.0,13.890000,0.000000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,0.0,19.760000,0.000000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,0.0,26.660000,0.000000,1.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,0.0,999.000000,18.000000,5.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [11]:
y

0        0
1        0
2        0
3        0
4        0
        ..
68812    0
68813    0
68814    0
68815    0
68816    0
Name: loan_status, Length: 68817, dtype: int64

In [12]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({0: 51366, 1: 246})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [13]:
# implement random oversampling
#    "low_risk": 0,
#    "high_risk":1

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({0: 51366, 1: 51366})

In [14]:
# Logistic regression using random oversampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [15]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,9971,7133
Actual 1,32,69


In [18]:
from sklearn.metrics import balanced_accuracy_score

#balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score :",balanced_accuracy_score(y_test, y_pred))

Balanced Accuracy Score : 0.6330656832053645


In [19]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))
#    "low_risk": 0,
#    "high_risk":1

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.58      0.68      0.74      0.63      0.39     17104
          1       0.01      0.68      0.58      0.02      0.63      0.40       101

avg / total       0.99      0.58      0.68      0.73      0.63      0.39     17205



### SMOTE Oversampling

In [20]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({0: 51366, 1: 51366})

In [21]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [22]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
print("Balanced Accuracy Score :",balanced_accuracy_score(y_test, y_pred))

Balanced Accuracy Score : 0.6228981235354591


In [25]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,11486,5618
Actual 1,43,58


In [26]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))
#    "low_risk": 0,
#    "high_risk":1

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.67      0.57      0.80      0.62      0.39     17104
          1       0.01      0.57      0.67      0.02      0.62      0.38       101

avg / total       0.99      0.67      0.57      0.80      0.62      0.39     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [27]:
# Fit the data using `ClusterCentroids` and check the count of each class
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 246, 1: 246})

In [28]:
# Logistic regression using cluster centroid undersampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=78)

In [30]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,7145,9959
Actual 1,34,67


In [31]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
print("Balanced Accuracy Score :",balanced_accuracy_score(y_test, y_pred))

Balanced Accuracy Score : 0.5405524386629496


In [32]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))
#    "low_risk": 0,
#    "high_risk":1

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.42      0.66      0.59      0.53      0.27     17104
          1       0.01      0.66      0.42      0.01      0.53      0.28       101

avg / total       0.99      0.42      0.66      0.59      0.53      0.27     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [33]:
# Use the SMOTEENN technique to perform combination sampling on the data
# Count the resampled classes
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({0: 62011, 1: 68460})

In [34]:
# Fit a Logistic regression model using random undersampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [35]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,9591,7513
Actual 1,24,77


In [36]:
# Calculate the Balanced Accuracy Score
from sklearn.metrics import balanced_accuracy_score
print("Balanced Accuracy Score :",balanced_accuracy_score(y_test, y_pred))

Balanced Accuracy Score : 0.6615611309727792


In [37]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))
#    "low_risk": 0,
#    "high_risk":1

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.56      0.76      0.72      0.65      0.42     17104
          1       0.01      0.76      0.56      0.02      0.65      0.44       101

avg / total       0.99      0.56      0.76      0.71      0.65      0.42     17205



# BalancedRandomForestClassifier

In [38]:
from sklearn.preprocessing import StandardScaler
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [40]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=0)

In [41]:
y_pred = brf.predict(X_test_scaled)

In [48]:
print("Balanced Accuracy Score :",balanced_accuracy_score(y_test, y_pred))

Balanced Accuracy Score : 0.7150021070862933


In [43]:
brf.feature_importances_ 

array([0.01314321, 0.03077901, 0.01700844, 0.00313162, 0.01905   ,
       0.00397255, 0.        , 0.01945129, 0.00533781, 0.00517056,
       0.00884203, 0.0020024 , 0.01509401, 0.01231961, 0.00136869,
       0.01616951, 0.01406281, 0.05527824, 0.0535062 , 0.05845277,
       0.06410496, 0.00798912, 0.        , 0.        , 0.06689966,
       0.00019683, 0.        , 0.00307648, 0.        , 0.0032897 ,
       0.01497101, 0.00726133, 0.00866974, 0.00649017, 0.00785031,
       0.01473839, 0.01289389, 0.01341524, 0.00545091, 0.00646039,
       0.01898344, 0.01413968, 0.01571343, 0.0093075 , 0.00650043,
       0.00901566, 0.01308422, 0.01556598, 0.01625994, 0.01451273,
       0.        , 0.        , 0.01292578, 0.01457285, 0.01356311,
       0.01349074, 0.0060954 , 0.01234757, 0.01741902, 0.00264764,
       0.00856588, 0.00937067, 0.00683626, 0.01168153, 0.01120301,
       0.0100701 , 0.01010005, 0.00784285, 0.00868861, 0.        ,
       0.        , 0.00182707, 0.0089095 , 0.01137117, 0.00885

In [44]:
# We can sort the features by their importance.
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.0668996611639256, 'last_pymnt_amnt'),
 (0.06410496066164578, 'total_rec_int'),
 (0.05845276932799135, 'total_rec_prncp'),
 (0.05527823583296287, 'total_pymnt'),
 (0.05350619908368669, 'total_pymnt_inv'),
 (0.030779008973783422, 'int_rate'),
 (0.01945129049533952, 'dti'),
 (0.019049995340207324, 'annual_inc'),
 (0.018983444609118813, 'max_bal_bc'),
 (0.017419016868501218, 'mths_since_recent_inq'),
 (0.017008441825951157, 'installment'),
 (0.016494980950080056, 'total_bal_ex_mort'),
 (0.016259941612615885, 'bc_open_to_buy'),
 (0.01616950620726373, 'out_prncp'),
 (0.015815068012675352, 'tot_hi_cred_lim'),
 (0.015713426457891422, 'total_rev_hi_lim'),
 (0.0155659773824032, 'avg_cur_bal'),
 (0.015094005610081727, 'revol_bal'),
 (0.014971011987358743, 'tot_cur_bal'),
 (0.014738388637317858, 'mths_since_rcnt_il'),
 (0.01457285103186157, 'mo_sin_old_rev_tl_op'),
 (0.014512734721787513, 'bc_util'),
 (0.014139684917533141, 'all_util'),
 (0.01406280836949891, 'out_prncp_inv'),
 (0.013724842886

In [45]:
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,14298,2806
Actual 1,41,60


In [46]:
# Calculate the Balanced Accuracy Score
from sklearn.metrics import balanced_accuracy_score

print("Balanced Accuracy Score :",balanced_accuracy_score(y_test, y_pred))

Balanced Accuracy Score : 0.7150021070862933


In [49]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))
#    "low_risk": 0,
#    "high_risk":1

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.84      0.59      0.91      0.70      0.51     17104
          1       0.02      0.59      0.84      0.04      0.70      0.48       101

avg / total       0.99      0.83      0.60      0.90      0.70      0.51     17205



# EasyEnsembleClassifier

In [50]:
from imblearn.ensemble import EasyEnsembleClassifier

In [51]:
eec = EasyEnsembleClassifier(random_state=42)

In [52]:
eec.fit(X_train, y_train)

EasyEnsembleClassifier(random_state=42)

In [53]:
 y_pred = eec.predict(X_test)

In [54]:
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,13902,3202
Actual 1,34,67


In [55]:
print("Balanced Accuracy Score :",balanced_accuracy_score(y_test, y_pred))

Balanced Accuracy Score : 0.738079332956682


In [56]:
print(classification_report_imbalanced(y_test, y_pred))
#    "low_risk": 0,
#    "high_risk":1

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.81      0.66      0.90      0.73      0.55     17104
          1       0.02      0.66      0.81      0.04      0.73      0.53       101

avg / total       0.99      0.81      0.66      0.89      0.73      0.55     17205

